In [282]:
import csv
import pandas
import math
import numpy as np
import random
from random import shuffle

def normalize(dataset):
    asd = np.array(dataset)
    for i in range(1,3):
        asd[:,i]=list(map(lambda x: (x-max(dataset.values[:,i]))/(max(dataset.values[:,i])-min(dataset.values[:,i])), dataset.values[:,i]))
    return asd

def logLikelihood(omega,dataset):
    df= pandas.DataFrame(dataset) 
    a = np.array(np.zeros(((dataset).shape[0])))
    d = np.array(np.zeros(((dataset).shape[0])))
    y = df.values[:,0]
    for i in range(df.shape[0]-1):
        for j in range(df.shape[1]-2):
            a[j] = omega[j]*df.values[i,j]
        d[i] = y[i]*(omega[0]+a.sum())-np.log(1+np.exp(omega[0]+a.sum()))
    return d.sum()
        
def parameter(omega,dataset):
    asd = np.array(np.zeros(((dataset).shape[0])))
    d= np.array(omega)
    a = np.array(np.zeros(((dataset).shape[0])))
    y = pandas.DataFrame(dataset).values[:,0]
    for j in range(pandas.DataFrame(dataset).shape[1]-1):
        for i in range(pandas.DataFrame(dataset).shape[0]-1):
            a[j] = omega[j]*pandas.DataFrame(dataset).values[i,j]
            if j==0:
                asd[i] = y[i]-(np.exp(omega[0]+a.sum())/(1+np.exp(omega[0]+a.sum())))
            else:
                asd[i] = pandas.DataFrame(dataset).values[i,j+1]*(y[i]-((np.exp(omega[0]+a.sum()))/(1+np.exp(omega[0]+a.sum()))))
            d[j]=asd.sum()
    return d

def score(omega,dataset):
    df= dataset
    scr=np.array(np.zeros(((dataset).shape[0])))
    for i in range(df.shape[0]-1):
        scr[i]=omega[0]+df[i,1]*omega[1]+df[i,2]*omega[2]+df[i,3]*omega[3]
    return np.exp(scr)/(1+np.exp(scr))

def scoreEvaluation(score,minimum):
    scr=np.array(np.zeros((len(score))))                      
    for i in range(len(score)):
        if score[i]<minimum:
            scr[i]=0
        else:
            scr[i]=1
    return scr
def evaluatePrediction(test,train):
    x = pandas.DataFrame(np.zeros(test.shape[0]-1))
    for i in range(test.shape[0]-1):
        x.values[i]=(test[i,0]==train[i]).astype(int)
    return x

def gradientAscent(omega,rate,dataset,iterations):
    for i in range(iterations):#while  iters < max_iters: # all(i >= precision for i in previous_step_size) and
        #  and  previous_step_size[1] > precision and previous_step_size[2] > precision and previous_step_size[3] > precision  and
        prev_omega = omega #Store current x value in prev_x
        omega = prev_omega + rate * parameter(prev_omega,dataset)  #Grad ascent
        #iters = iters+1 #iteration count
        #print(omega)
    return omega

#handle data
titanic_Data=pandas.read_csv('titanicdata.csv')#,skiprows=lambda x: x not in pandas.Series(range(1,800)))
titanic_Data=normalize(titanic_Data)
number_of_rows = titanic_Data.shape[0]
index = list(np.arange(number_of_rows))
np.random.shuffle(index)
random_data=titanic_Data
for l in range(len(index)):
    random_data[l,:]=titanic_Data[index[l],:]
titanic_Data=random_data
train_data=np.take(titanic_Data, range(round(titanic_Data.shape[0]*0.6)), axis=0)
valid_data=np.take(titanic_Data, range(round(titanic_Data.shape[0]*0.6),round(titanic_Data.shape[0]*0.6)+round(titanic_Data.shape[0]*0.2)), axis=0)
test_data=np.take(titanic_Data, range(round(titanic_Data.shape[0]*0.8),round(titanic_Data.shape[0]*0.6)+round(titanic_Data.shape[0]*0.2)+round(titanic_Data.shape[0]*0.2)), axis=0)
#test_data
omega=[0,0,0,0] #initial values for omega
omega=gradientAscent(omega,0.00001,train_data,190)
scor=score(omega,test_data)
prediction=scoreEvaluation(scor,0.5)
evaluation=evaluatePrediction(test_data,prediction)
number_of_equal_elements = np.sum(evaluation)
total_elements = (evaluation.shape[0])
percentage = number_of_equal_elements[0]/total_elements
number_of_class=test_data.shape[1]-1
print('total number of elements: \t\t{}'.format(total_elements))
print('total number of classes: \t\t{}'.format(number_of_class))
print('number of identical elements: \t\t{}'.format(number_of_equal_elements[0]))
print('number of different elements: \t\t{}'.format(total_elements-number_of_equal_elements[0]))
print('percentage of identical elements: \t{:.2f}%'.format(percentage*100))

total number of elements: 		177
total number of classes: 		3
number of identical elements: 		140.0
number of different elements: 		37.0
percentage of identical elements: 	79.10%


array([ -0.30259989,  -0.88638236, -76.82101482,   0.        ])

total number of elements: 		177
total number of classes: 		3
number of identical elements: 		119.0
number of different elements: 		58.0
percentage of identical elements: 	67.23%
